In [2]:
#import codecs
import os
from tqdm import tqdm
#import re
#import math
#from operator import itemgetter, attrgetter
os.chdir(r"C:\Users\AGN's SP\OneDrive\课件\自然语言处理\HMM")

In [14]:
# 所有词语
all_words = []
# 所有的词性
all_poses = []
# 每个词性出现的频率
fre = {}
# 先验概率矩阵
pi = {}
# 状态转移概率矩阵
A = {}
# 观测概率矩阵
B = {}
# dp概率
dp = []
# 路径记录
pre = []
with open(r"人民日报标记语料库及部分验证数据示例\199801.txt") as f:
    texts = f.readlines()
try:
    with tqdm(range(len(texts))) as t:
        for i in t:
            t.update()
            text = texts[i]
            #print(text)
            if text.strip() == '':
                continue
            tmp = text.strip().split()
            #按空格切分
            n = len(tmp)
            for m in range(n):
                #切分词和他的词性
                word = tmp[m].split('/')
                
                if '[' in word[0]: #去除部分有[]的词
                    word[0] = word[0].split('[')[1]
                if ']' in word[1]:
                    word[1] = word[1].split(']')[0]
                    
                if (word[0] not in all_words):#添加新出现的单词
                    all_words.append(word[0])
                if (word[1] not in all_poses):#添加新出现的词性
                    all_poses.append(word[1])
                    
                pre=tmp[m-1].split('/')   #标记上一个单词
                try:
                    fre[word[1]]+=1       #词性的出现频率+1
                except:
                    fre[word[1]]=1
                if(m==1):                #第一个词的词性的频率
                    try:
                        pi[word[1]]+=1
                    except:
                        pi[word[1]]=1
                elif(m > 0):             #从前一个单词的词性到本单词词性的次数+1
                    try:
                        A[pre[1]]
                        try:
                            A[pre[1]][word[1]]+=1
                        except:
                            A[pre[1]][word[1]]=1
                    except:
                        A[pre[1]]={}
                        A[pre[1]][word[1]]=1
                try:                   #从本一个单词的词性到本单词的次数+1
                    B[word[1]]
                    try:
                        B[word[1]][word[0]]+=1
                    except:
                        B[word[1]][word[0]]=1
                except:
                    B[word[1]]={}
                    B[word[1]][word[0]]=1
except KeyboardInterrupt:
    t.close()
    raise
t.close()
#print(A)
ca={}
cb={}
for i in all_poses:
    ca[i]=0
    cb[i]=0
    try:
        pi[i]=pi[i]*1.0/len(texts)   #将次数变成概率
    except:
        pi[i] = 0
    for j in all_poses:
        if j not in A[i].keys():    #在一个词性通往下一个词性，有词性未在文章在出现，即没有从A词性到B词性的时候，给一个平滑0.5的值，并且要将A的总次数+1
            ca[i]+=1
            A[i][j]=0.5
    for j in all_words:            #在一个词性通往一个单词，有单词未在文章在出现，即没有从A词性到a单词的时候，给一个平滑0.5的值，并且要将B的总次数+1
        if j not in B[i].keys():
            cb[i]+=1
            B[i][j]=0.5
#除次数变概率
for i in all_poses:
    for j in all_poses:
        A[i][j]=A[i][j]*1.0/(fre[i]+ca[i])
    for j in all_words:
        B[i][j]=B[i][j]*1.0/(fre[i]+cb[i])

print("语料库训练结束")

100%|████████████████████████████████████████████████████████████████████████████| 23064/23064 [07:08<00:00, 53.83it/s]


语料库训练结束


In [17]:
while(True):
    tmp=input("请输入需要词性标注的句子，以空格分割: ")
    if tmp=='':
        break
    text=tmp.split()
    num=len(text)
    wbt=[{} for i in range(0,num)]  #维比特算法
    pre=[{} for i in range(0,num)]
    #初始化
    for k in all_poses:
        for j in range(0,num):
            wbt[j][k]=0
            pre[j][k]=""
    n=len(all_poses)
    for c in all_poses:
        if text[0] in B[c].keys():
            wbt[0][c]=pi[c]*B[c][text[0]]
        else:
            wbt[0][c]=pi[c]*0.5/(cb[c]+fre[c]) #若不存在该词也给一个权重
    for i in range(1,num):
        for j in all_poses:
            for k in all_poses:
                b=0
                if text[i] in B[j].keys():
                    b=B[j][text[i]]
                else:
                    b=0.5/(cb[j]+fre[j])  #若不存在该词也给一个权重
                if(wbt[i][j]<wbt[i-1][k]*A[k][j]*b):
                    wbt[i][j]=wbt[i-1][k]*A[k][j]*b
                    pre[i][j]=k       

    MAX=""
    for j in all_poses:
        if(MAX=="" or wbt[num-1][j]>wbt[num-1][MAX]):  #选出概率最大的一串序列
            MAX=j

    if(wbt[num-1][MAX]==0):            
        print( "您的句子也太难太长了，我不行了！您自己来！")
        continue
    res={}    
    i=num-1
    while(i>=0): #通过逆向传播，找到每个位置的词性
        res[i]=MAX
        MAX=pre[i][MAX]
        i-=1

    print(' '.join([(text[i])+"\\"+res[i] for i in range(num)]))



请输入需要词性标注的句子，以空格分割: 本报  驻  俄罗斯  记者  刘  刚   俄罗斯  货币  改革  已  数  日  了  。  总的看  ，  流通领域  的  工作  按部就班  ，  井井有条  ；  百姓  对待  货币  改革  态度  积极  ，  心情  平静  。  与  前  几  次  货币  改革  相比  ，  这次  改革  恐怕  是  进行  得  最  平稳  的  一  次  。   俄罗斯  中央  银行  行长  杜比宁  认为  ，  货币  改革  是  反  通货膨胀  的  重要  手段  ，  将  为  经济  增长  创造  有利  条件  。  从  目前  的  经济  形势  看  ，  实施  货币  改革  的  条件  已经  成熟  。  经济  专家  认为  ，  俄  经济  从  １９９７年  起  开始  走  出  谷底  并  略  呈  上升  之  势  。  连续  多年  的  通货膨胀  得到  有效  的  控制  ，  卢布  汇率  基本  稳定  。  去年  工业  产值  增长  ２％  ，  国内  生产总值  增长  了  ０．４％  ，  尽管  增长  幅度  不  大  ，  但  这  是  最近  ７  年  首  次  出现  增长  的  局面  。  据  报道  ，  俄  外汇  储备  增长  速度  也  很  快  ，  至  去年底  已  达  １８０亿  美元  。   从  今年  １月  １日  起  ，  俄罗斯  新  卢布  已  正式  在  市场  上  流通  ，  与此同时  ，  旧币  仍  继续  使用  ，  其  比价  为  ：  １  新  卢布  １０００  旧  卢布  。  这个  决定  自  去年  公布  之  日  起  ，  政府  做  了  大量  宣传  解释  工作  ，  几乎  每天  都  可以  在  报纸  和  电视  上  看到  有关  货币  改革  的  节目  和  文章  。  据  俄罗斯  《  消息报  》  去年  １２月  ３１日  报道  ，  俄  中央  银行  连日来  繁忙  的  程度  不可名状  ，  全体  员工